In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_ = pd.read_csv("/Users/ibrahimyucel/Downloads/Gain Enerji Intern Analyst Case Study_2025/data/out/YANBOLU_HES_birlesik_2024.csv")
df = df_.copy()

In [28]:
print("=== Veri Kalite Kontrolü ===")
print(f"Toplam satır: {len(df)}")
print(f"Eksik değerler: {df.isnull().sum().sum()}")
print(f"Duplikat satırlar: {df.duplicated().sum()}")

# Saat kontrolü (24 saat × 366 gün = 8784)
print(f"\n=== Saat Dağılımı ===")
print(f"Benzersiz saat sayısı: {df['Saat'].nunique()}")
print(f"Benzersiz tarih sayısı: {df['Tarih'].nunique()}")

# Değer aralıkları
print(f"\n=== Değer Aralıkları ===")
print(f"PTF: {df['PTF'].min():.2f} - {df['PTF'].max():.2f}")
print(f"SMF: {df['SMF'].min():.2f} - {df['SMF'].max():.2f}")
print(f"KGÜP: {df['KGUP'].min():.2f} - {df['KGUP'].max():.2f}")
print(f"Üretim: {df['URETIM'].min():.2f} - {df['URETIM'].max():.2f}")

# Dengesizlik maliyeti özeti
print(f"\n=== Dengesizlik Maliyeti ===")
print(f"Toplam DM: {df['Dengesizlik_Maliyeti'].sum():.2f} TL")
print(f"Ortalama Birim DM: {df['Birim_DM'].mean():.4f} TL/MWh")
print(f"Toplam Üretim: {df['URETIM'].sum():.2f} MWh")

=== Veri Kalite Kontrolü ===
Toplam satır: 8784
Eksik değerler: 0
Duplikat satırlar: 0

=== Saat Dağılımı ===
Benzersiz saat sayısı: 24
Benzersiz tarih sayısı: 366

=== Değer Aralıkları ===
PTF: 0.00 - 3000.00
SMF: 0.00 - 3000.00
KGÜP: 0.00 - 8.50
Üretim: 0.00 - 8.77

=== Dengesizlik Maliyeti ===
Toplam DM: 1462906.77 TL
Ortalama Birim DM: 64.7599 TL/MWh
Toplam Üretim: 21374.56 MWh


In [4]:
# Aylık agregasyon ve temel performans metrikleri
df_monthly = df.groupby('Ay').agg({
    'KGUP': 'sum',
    'URETIM': 'sum', 
    'Dengesizlik': 'sum',
    'GOP_Geliri': 'sum',
    'Dengesizlik_Tutarı': 'sum',
    'Net_Gelir': 'sum',
    'Dengesizlik_Maliyeti': 'sum'
}).round(2)

# Aylık performans metrikleri
df_monthly['Dengesizlik_Orani'] = (df_monthly['Dengesizlik'] / df_monthly['KGUP'] * 100).round(4)
df_monthly['Birim_DM_Aylik'] = (df_monthly['Dengesizlik_Maliyeti'] / df_monthly['URETIM']).round(4)

# Kapasite faktörü hesaplama (düzeltilmiş)
days_per_month = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]  # 2024 artık yıl
max_possible_production = [8.5 * 24 * days for days in days_per_month]
df_monthly['Kapasite_Faktoru'] = (df_monthly['URETIM'] / max_possible_production * 100).round(2)

df_monthly['DM_Gelir_Orani'] = (df_monthly['Dengesizlik_Maliyeti'] / df_monthly['Net_Gelir'] * 100).round(4)

print("=== MELKOM HES - Aylık Performans Özeti ===")
df_monthly

=== MELKOM HES - Aylık Performans Özeti ===


,KGUP,URETIM,Dengesizlik,GOP_Geliri,Dengesizlik_Tutarı,Net_Gelir,Dengesizlik_Maliyeti,Dengesizlik_Orani,Birim_DM_Aylik,Kapasite_Faktoru,DM_Gelir_Orani
Ay,,,,,,,,,,,
1,830.70,871.77,41.07,1584779.84,31607.53,1616387.37,100554.50,4.9440,115.3452,13.79,6.2209
2,1142.80,1229.06,86.26,2144969.13,80917.21,2225886.34,109639.00,7.5481,89.2056,20.78,4.9256
3,1896.30,2163.01,266.71,4076774.95,422254.64,4499029.59,164604.32,14.0648,76.0997,34.20,3.6587
4,4747.50,4953.92,206.42,8505805.58,67209.72,8573015.30,268457.98,4.3480,54.1910,80.95,3.1314
5,4601.80,4883.06,281.26,9626177.22,207846.60,9834023.82,268476.66,6.1120,54.9812,77.21,2.7301
6,1221.30,1155.80,-65.50,2719159.20,-211631.25,2507527.94,61627.71,-5.3631,53.3204,18.89,2.4577
7,1250.70,1398.59,147.89,3335827.74,293581.63,3629409.38,92238.54,11.8246,65.9511,22.12,2.5414
8,310.40,271.52,-38.88,798706.88,-133569.47,665137.41,25732.01,-12.5258,94.7702,4.29,3.8687
9,565.10,550.54,-14.56,1383181.59,-117213.86,1265967.73,63497.25,-2.5765,115.3363,9.00,5.0157


In [7]:
# Ceza asimetrisi stratejisi (overweight optimizasyonu)
print("=== YANBOLU HES - Ceza Asimetrisi Stratejisi (Overweight Optimizasyonu) ===")

# Pozitif ve negatif dengesizliklerin ceza oranlarını analiz et
positive_imb = df[df['Dengesizlik'] > 0].copy()
negative_imb = df[df['Dengesizlik'] < 0].copy()

if len(positive_imb) > 0 and len(negative_imb) > 0:
    # Ceza oranları (Dengesizlik Tutarı / (Dengesizlik * PTF))
    positive_imb['Penalty_Rate'] = positive_imb['Dengesizlik_Tutarı'] / (positive_imb['Dengesizlik'] * positive_imb['PTF'])
    negative_imb['Penalty_Rate'] = abs(negative_imb['Dengesizlik_Tutarı']) / (abs(negative_imb['Dengesizlik']) * negative_imb['PTF'])
    
    avg_pos_penalty = positive_imb['Penalty_Rate'].mean()
    avg_neg_penalty = negative_imb['Penalty_Rate'].mean()
    
    print(f"Ortalama Pozitif Dengesizlik Ceza Oranı: {avg_pos_penalty:.3f}")
    print(f"Ortalama Negatif Dengesizlik Ceza Oranı: {avg_neg_penalty:.3f}")
    print(f"Ceza Asimetrisi (Neg/Pos): {avg_neg_penalty/avg_pos_penalty:.2f}")
    
    # Overweight stratejisi simülasyonu
    overweight_scenarios = [0.05, 0.10, 0.15, 0.20]  # %5, %10, %15, %20 fazla planlama
    
    print(f"\nOverweight Stratejisi Simülasyonu:")
    for ow in overweight_scenarios:
        sim_kgup = df['KGUP'] * (1 + ow)
        sim_dengesizlik = df['URETIM'] - sim_kgup
        
        # Yeni dengesizlik tutarları (vectorized çözüm)
        sim_dt = np.where(
            sim_dengesizlik >= 0,
            sim_dengesizlik * df['Poz_DF'],
            sim_dengesizlik * df['Neg_DF']
        )
        
        # Dengesizlik maliyeti değişimi
        original_dm = df['Dengesizlik_Maliyeti'].sum()
        sim_dm = ((df['URETIM'] * df['PTF']) - (sim_kgup * df['PTF'] + sim_dt)).clip(lower=0).sum()
        
        dm_change = sim_dm - original_dm
        dm_change_pct = (dm_change / original_dm) * 100
        
        print(f"  {ow*100:2.0f}% Overweight: DM Değişimi = {dm_change:,.0f} TL ({dm_change_pct:+.1f}%)")

=== YANBOLU HES - Ceza Asimetrisi Stratejisi (Overweight Optimizasyonu) ===
Ortalama Pozitif Dengesizlik Ceza Oranı: 0.841
Ortalama Negatif Dengesizlik Ceza Oranı: 1.277
Ceza Asimetrisi (Neg/Pos): 1.52

Overweight Stratejisi Simülasyonu:
   5% Overweight: DM Değişimi = 2,376 TL (+0.2%)
  10% Overweight: DM Değişimi = 98,525 TL (+6.7%)
  15% Overweight: DM Değişimi = 226,312 TL (+15.5%)
  20% Overweight: DM Değişimi = 385,287 TL (+26.3%)


In [8]:
print("=== YANBOLU HES - Risk Analizi ===")

# En yüksek DM yaşanan günler
daily_dm = df.groupby('Tarih')['Dengesizlik_Maliyeti'].sum()
print("En yüksek dengesizlik maliyeti yaşanan 10 gün:")
# En yüksek dengesizlik maliyeti yaşanan 10 günü, önce DM'ye göre en yüksekten seçip, sonra tarihe göre (en yeni gün en üstte) göstermek için:
top10_dm_days = daily_dm.sort_values(ascending=False).head(10)
top10_dm_days = top10_dm_days.sort_index(ascending=False)
print(top10_dm_days)

hourly_analysis = df.groupby('Saat').agg({
    'Dengesizlik_Maliyeti': 'mean',
    'Dengesizlik': 'mean',
    'PTF': 'mean',
    'KGUP': 'mean'
}).reset_index()

# Dengesizlik oranı = dengesizlik / planlanan üretim
hourly_analysis['Dengesizlik_Orani'] = (
    hourly_analysis['Dengesizlik'] / hourly_analysis['KGUP'] * 100
).round(2)

# En riskli saatler
risky_hours = hourly_analysis.nlargest(5, 'Dengesizlik_Orani')[['Dengesizlik_Maliyeti', 'Dengesizlik_Orani', 'PTF']]
print(f"\nEn riskli 5 saat:")
print(risky_hours)

# Mevsimsel risk analizi
seasonal_risk = df.groupby('Ay').agg({
    'Dengesizlik_Maliyeti': ['sum', 'mean', 'std'],
    'Birim_DM': 'mean'
}).round(2)
seasonal_risk.columns = ['Toplam_DM', 'Ortalama_DM', 'DM_Volatilite', 'Birim_DM']
seasonal_risk = seasonal_risk.sort_values('Birim_DM', ascending=False)
print(f"\nMevsimsel Risk Profili (Birim_DM'ye göre büyükten küçüğe sıralı):")
print(seasonal_risk)

# Extreme event analizi
extreme_dm = df[df['Dengesizlik_Maliyeti'] > df['Dengesizlik_Maliyeti'].quantile(0.95)]
print(f"\nExtreme DM Events (>95. persentil):")
print(f"Toplam olay sayısı: {len(extreme_dm)}")
print(f"Bu olayların ortalama PTF'si: {extreme_dm['PTF'].mean():.0f} TL/MWh")
print(f"Bu olayların ortalama dengesizlik büyüklüğü: {extreme_dm['Dengesizlik'].abs().mean():.2f} MWh")

=== YANBOLU HES - Risk Analizi ===
En yüksek dengesizlik maliyeti yaşanan 10 gün:
Tarih
2024-10-19    21108.767868
2024-05-27    46263.721323
2024-05-25    23673.529019
2024-05-12    24459.343656
2024-04-10    60509.518570
2024-04-07    48457.112252
2024-04-06    30616.446322
2024-03-29    43394.518223
2024-02-07    53403.528660
2024-01-17    25482.239243
Name: Dengesizlik_Maliyeti, dtype: float64

En riskli 5 saat:
    Dengesizlik_Maliyeti  Dengesizlik_Orani          PTF
20            150.582335               9.25  2726.846175
21            126.198992               8.82  2652.683907
22            183.751810               7.22  2416.326557
3             151.390737               6.30  2002.855601
0             146.195957               6.17  2457.293142

Mevsimsel Risk Profili (Birim_DM'ye göre büyükten küçüğe sıralı):
    Toplam_DM  Ortalama_DM  DM_Volatilite  Birim_DM
Ay                                                 
1   100554.50       135.15         371.79    118.55
10  142035.14  

In [10]:
# En maliyetli 30 saat detay analizi
print("=== YANBOLU HES - En Maliyetli 30 Saat Detay Analizi ===")

# En yüksek dengesizlik maliyeti olan 30 saati seç
top_30_cost = df.nlargest(30, 'Dengesizlik_Maliyeti').copy()
top_30_cost['PTF_SMF_Spread'] = abs(top_30_cost['PTF'] - top_30_cost['SMF'])

print(f"En Maliyetli 30 Saat - Genel İstatistikler:")
print(f"Toplam Maliyet: {top_30_cost['Dengesizlik_Maliyeti'].sum():,.0f} TL")
print(f"Ortalama Maliyet: {top_30_cost['Dengesizlik_Maliyeti'].mean():,.0f} TL/saat")
print(f"Toplam Maliyetteki Payı: {top_30_cost['Dengesizlik_Maliyeti'].sum() / df['Dengesizlik_Maliyeti'].sum() * 100:.1f}%")

# Zaman dağılımı
time_distribution = top_30_cost.groupby('Saat').size().sort_values(ascending=False)
print(f"\nEn Maliyetli Saatlerin Zaman Dağılımı:")
print(time_distribution.head())

month_distribution = top_30_cost.groupby('Ay').size().sort_values(ascending=False)
print(f"\nEn Maliyetli Saatlerin Ay Dağılımı:")
print(month_distribution.head())

# Ortak karakteristikler
print(f"\nEn Maliyetli 30 Saatin Karakteristikleri:")
characteristics = top_30_cost.agg({
    'PTF': ['mean', 'min', 'max'],
    'SMF': ['mean', 'min', 'max'],
    'URETIM': ['mean', 'min', 'max'],
    'KGUP': ['mean', 'min', 'max'],
    'Dengesizlik': ['mean', 'min', 'max'],
    'PTF_SMF_Spread': ['mean', 'min', 'max'] if 'PTF_SMF_Spread' in df.columns else ['mean']
}).round(2)

print(characteristics)

# Bu saatlerdeki dengesizlik yönü
positive_imbalance = (top_30_cost['Dengesizlik'] > 0).sum()
negative_imbalance = (top_30_cost['Dengesizlik'] < 0).sum()
print(f"\nDengesizlik Yönü Dağılımı:")
print(f"Pozitif (Fazla Üretim): {positive_imbalance} saat")
print(f"Negatif (Az Üretim): {negative_imbalance} saat")

# Detaylı liste (ilk 10 saat)
print(f"\nEn Maliyetli 10 Saat Detayı:")
detailed_top10 = top_30_cost.head(10)[['Tarih', 'Saat', 'PTF', "SMF", 'KGUP', 'URETIM', 'Dengesizlik', 'Dengesizlik_Maliyeti']]
detailed_top10

=== YANBOLU HES - En Maliyetli 30 Saat Detay Analizi ===
En Maliyetli 30 Saat - Genel İstatistikler:
Toplam Maliyet: 153,451 TL
Ortalama Maliyet: 5,115 TL/saat
Toplam Maliyetteki Payı: 10.5%

En Maliyetli Saatlerin Zaman Dağılımı:
Saat
16:00    5
23:00    2
15:00    2
05:00    2
20:00    2
dtype: int64

En Maliyetli Saatlerin Ay Dağılımı:
Ay
4     15
5      7
1      2
12     2
2      1
dtype: int64

En Maliyetli 30 Saatin Karakteristikleri:
          PTF      SMF  URETIM  KGUP  Dengesizlik  PTF_SMF_Spread
mean  1761.17  1186.58    5.80  3.72         2.08         1446.44
min     51.01   425.00    0.00  0.00        -4.11             NaN
max   2800.00  2999.00    8.59  8.00         6.05             NaN

Dengesizlik Yönü Dağılımı:
Pozitif (Fazla Üretim): 23 saat
Negatif (Az Üretim): 7 saat

En Maliyetli 10 Saat Detayı:


,Tarih,Saat,PTF,SMF,KGUP,URETIM,Dengesizlik,Dengesizlik_Maliyeti
2343,2024-04-07,15:00,81.00,2400.00,5.0,1.60,-3.40,8129.400000
2344,2024-04-07,16:00,129.98,2400.00,5.0,1.56,-3.44,8056.548800
2342,2024-04-07,14:00,51.01,2400.00,5.0,2.08,-2.92,7069.290800
2420,2024-04-10,20:00,2525.00,750.00,4.0,7.55,3.55,6381.125000
2401,2024-04-10,01:00,2100.00,700.00,4.0,8.40,4.40,6252.400000
3536,2024-05-27,08:00,2199.68,1000.00,2.5,7.57,5.07,6234.477600
2345,2024-04-07,17:00,750.00,2400.00,5.0,1.45,-3.55,6113.100000
2402,2024-04-10,02:00,1999.99,675.00,4.0,8.38,4.38,5892.151200
3496,2024-05-25,16:00,293.77,1633.39,6.0,1.89,-4.11,5707.235187
2320,2024-04-06,16:00,750.01,2640.00,8.0,5.15,-2.85,5612.191500


In [14]:
def kpi_summary(df_monthly, plant_name="Santral"):
    num_cols = df_monthly.select_dtypes(include='number').columns

    summary = pd.DataFrame({
        "SUM": df_monthly[num_cols].sum(),
        "MEAN": df_monthly[num_cols].mean()
    }).round(2)

    summary.index.name = f"{plant_name} - KPI"
    return summary

yanbolu_summary = kpi_summary(df_monthly, "YANBOLU_HES")
yanbolu_summary

,SUM,MEAN
YANBOLU_HES - KPI,,
KGUP,20537.75,1711.48
URETIM,21374.56,1781.21
Dengesizlik,836.81,69.73
GOP_Geliri,43800257.72,3650021.48
Dengesizlik_Tutarı,146340.68,12195.06
Net_Gelir,43946598.40,3662216.53
Dengesizlik_Maliyeti,1462906.77,121908.90
Dengesizlik_Orani,20.79,1.73
Birim_DM_Aylik,959.07,79.92


In [25]:
STEP = 0.5

def optimize_kgnp(df, riskli_aylar, riskli_saatler, use_mean=True):
    """
    Belirlenen ay ve saatlere göre KGUP optimizasyonu yapar
    ve özet rapor döner.
    """
    df_opt = df.copy()

    # Saat string → int
    if df_opt['Saat'].dtype == 'object':
        df_opt['Saat'] = pd.to_datetime(df_opt['Saat'], format='%H:%M').dt.hour

    df_opt['KGUP_Opt'] = df_opt['KGUP']

    # --- optimizasyon ---
    for idx, row in df_opt.iterrows():
        if (row['Saat'] in riskli_saatler) and (row['Ay'] in riskli_aylar):
            kgup = row['KGUP_Opt']
            uretim = row['URETIM']
            ptf = row['PTF']
            poz = row['Poz_DF']
            neg = row['Neg_DF']

            d = uretim - kgup  # dengesizlik

            if d < 0:  # eksik üretim
                if neg > ptf:
                    kgup -= STEP
            else:      # fazla üretim
                if poz < ptf:
                    kgup += STEP

            if kgup < 0:
                kgup = 0

            df_opt.at[idx, 'KGUP_Opt'] = kgup

    # --- gelir hesaplamaları ---
    df_opt['GOP_Opt'] = df_opt['KGUP_Opt'] * df_opt['PTF']
    df_opt['Dengesizlik_Opt'] = df_opt['URETIM'] - df_opt['KGUP_Opt']
    df_opt['Fiyat_Opt'] = df_opt.apply(
        lambda row: row['Poz_DF'] if row['Dengesizlik_Opt'] >= 0 else row['Neg_DF'], axis=1
    )
    df_opt['Dengesizlik_Tutarı_Opt'] = df_opt['Dengesizlik_Opt'] * df_opt['Fiyat_Opt']
    df_opt['Net_Opt'] = df_opt['GOP_Opt'] + df_opt['Dengesizlik_Tutarı_Opt']

    # --- toplam sonuç ---
    toplam_mevcut = df['Net_Gelir'].sum()
    toplam_yeni = df_opt['Net_Opt'].sum()
    iyilestirme = toplam_yeni - toplam_mevcut

    print("Mevcut Net Gelir:", toplam_mevcut)
    print("Optimize Net Gelir:", toplam_yeni)
    print("İyileştirme:", iyilestirme)

    # --- detay rapor ---
    ay_ad_tr = {
        1: 'ocak', 2: 'şubat', 3: 'mart', 4: 'nisan', 5: 'mayıs', 6: 'haziran',
        7: 'temmuz', 8: 'ağustos', 9: 'eylül', 10: 'ekim', 11: 'kasım', 12: 'aralık'
    }

    agg_fn = np.mean if use_mean else np.sum
    label = "ort" if use_mean else "top"

    for ay in riskli_aylar:
        for saat in riskli_saatler:
            mask = (df['Ay'] == ay) & (df['Saat'] == saat)
            if mask.sum() == 0:
                continue

            kgup_mevcut = agg_fn(df.loc[mask, 'KGUP'])
            uretim_mevcut = agg_fn(df.loc[mask, 'URETIM'])
            kgup_yeni = agg_fn(df_opt.loc[mask, 'KGUP_Opt'])
            net_mevcut = agg_fn(df.loc[mask, 'Net_Gelir'])
            net_yeni = agg_fn(df_opt.loc[mask, 'Net_Opt'])
            delta_kar = net_yeni - net_mevcut

            print(
                f"ay: {ay_ad_tr.get(ay, ay)} saat: {saat:02d}:00 "
                f"mevcut KGUP({label}): {kgup_mevcut:.2f} "
                f"mevcut uretim({label}): {uretim_mevcut:.2f} "
                f"yeni KGUP({label}): {kgup_yeni:.2f} "
                f"kar_yeni({label}): {net_yeni:,.0f} "
                f"kar_artis({label}): {delta_kar:,.0f}"
            )

    return df_opt

In [19]:
df_opt_risk = optimize_kgnp(df, riskli_aylar=[1, 2, 4, 5, 10, 11], riskli_saatler=[0,3,20,21,22])

Mevcut Net Gelir: 43946598.397250004
Optimize Net Gelir: 43972418.829670005
İyileştirme: 25820.432420000434


In [26]:
df_opt_maliyet = optimize_kgnp(df, riskli_aylar=[1, 2, 4, 5, 12], riskli_saatler=[5, 15, 16, 20, 23]) # maliyetli saatlerde KGUP optimizasyonu

Mevcut Net Gelir: 43946598.397250004
Optimize Net Gelir: 43983023.112712
İyileştirme: 36424.71546199918


In [4]:
STEP = 0.5
def optimize_kgnp_combined(df, risk_aylar, risk_saatler, maliyet_aylar, maliyet_saatler):
    """
    Kesişen zamanlar için risk stratejisi, kalanlar için maliyet stratejisi uygular
    """
    df_opt = df.copy()
    
    # Saat string → int
    if df_opt['Saat'].dtype == 'object':
        df_opt['Saat'] = pd.to_datetime(df_opt['Saat'], format='%H:%M').dt.hour
    
    df_opt['KGUP_Opt'] = df_opt['KGUP']
    df_opt['Strateji'] = 'Değişiklik_Yok'
    
    # Risk ve maliyet kümelerini tanımla
    risk_set = set((ay, saat) for ay in risk_aylar for saat in risk_saatler)
    maliyet_set = set((ay, saat) for ay in maliyet_aylar for saat in maliyet_saatler)
    
    # Kesişenleri bul
    kesisen_set = risk_set.intersection(maliyet_set)
    
    # Sadece risk setinde olanlar (kesişenler hariç)
    sadece_risk_set = risk_set - kesisen_set
    
    # Sadece maliyet setinde olanlar (kesişenler hariç)
    sadece_maliyet_set = maliyet_set - kesisen_set
    
    print(f"Kesişen zaman dilimleri: {len(kesisen_set)}")
    print(f"Sadece risk stratejisi: {len(sadece_risk_set)}")
    print(f"Sadece maliyet stratejisi: {len(sadece_maliyet_set)}")
    print(f"Toplam optimize edilecek kombinasyon: {len(kesisen_set) + len(sadece_risk_set) + len(sadece_maliyet_set)}")
    
    # Optimizasyon uygula
    for idx, row in df_opt.iterrows():
        ay_saat = (row['Ay'], row['Saat'])
        
        # Strateji uygulanacak mı kontrol et
        if ay_saat in kesisen_set or ay_saat in sadece_risk_set or ay_saat in sadece_maliyet_set:
            
            # Strateji adını belirle
            if ay_saat in kesisen_set or ay_saat in sadece_risk_set:
                 df_opt.at[idx, 'Strateji'] = 'Risk_Stratejisi'
            else: # sadece_maliyet_set
                 df_opt.at[idx, 'Strateji'] = 'Maliyet_Stratejisi'

            # Ortak optimizasyon mantığını uygula
            kgup = row['KGUP_Opt']
            uretim = row['URETIM']
            ptf = row['PTF']
            poz = row['Poz_DF']
            neg = row['Neg_DF']
            
            d = uretim - kgup
            
            if d < 0:  # eksik üretim
                if neg > ptf:
                    kgup -= STEP
            else:      # fazla üretim
                if poz < ptf:
                    kgup += STEP
            
            if kgup < 0:
                kgup = 0
            df_opt.at[idx, 'KGUP_Opt'] = kgup
    
    # Gelir hesaplamaları
    df_opt['GOP_Opt'] = df_opt['KGUP_Opt'] * df_opt['PTF']
    df_opt['Dengesizlik_Opt'] = df_opt['URETIM'] - df_opt['KGUP_Opt']
    df_opt['Fiyat_Opt'] = df_opt.apply(
        lambda row: row['Poz_DF'] if row['Dengesizlik_Opt'] >= 0 else row['Neg_DF'], axis=1
    )
    df_opt['Dengesizlik_Tutarı_Opt'] = df_opt['Dengesizlik_Opt'] * df_opt['Fiyat_Opt']
    df_opt['Net_Opt'] = df_opt['GOP_Opt'] + df_opt['Dengesizlik_Tutarı_Opt']
    
    # Sonuçlar
    toplam_mevcut = df['Net_Gelir'].sum()
    toplam_yeni = df_opt['Net_Opt'].sum()
    iyilestirme = toplam_yeni - toplam_mevcut
    
    print(f"\n=== SONUÇLAR ===")
    print(f"Mevcut Net Gelir: {toplam_mevcut:,.0f} TL")
    print(f"Optimize Net Gelir: {toplam_yeni:,.0f} TL")
    print(f"Toplam İyileştirme: {iyilestirme:,.0f} TL")
    
    # Strateji bazında detay
    strateji_detay = df_opt.groupby('Strateji').agg({
        'Net_Gelir': 'sum',
        'Net_Opt': 'sum'
    })
    strateji_detay['Iyilestirme'] = strateji_detay['Net_Opt'] - strateji_detay['Net_Gelir']
    
    print(f"\n=== STRATEJİ BAZINDA DETAY ===")
    print(strateji_detay)
    
    return df_opt

df_opt_combined = optimize_kgnp_combined(
    df, 
    risk_aylar=[1, 2, 4, 5, 10, 11], 
    risk_saatler=[0,3,20,21,22],
    maliyet_aylar=[1, 2, 4, 5, 12], 
    maliyet_saatler=[5, 15, 16, 20, 23]
)

Kesişen zaman dilimleri: 4
Sadece risk stratejisi: 26
Sadece maliyet stratejisi: 21
Toplam optimize edilecek kombinasyon: 51

=== SONUÇLAR ===
Mevcut Net Gelir: 43,946,598 TL
Optimize Net Gelir: 44,004,449 TL
Toplam İyileştirme: 57,851 TL

=== STRATEJİ BAZINDA DETAY ===
                       Net_Gelir       Net_Opt  Iyilestirme
Strateji                                                   
Değişiklik_Yok      3.259358e+07  3.259358e+07      0.00000
Maliyet_Stratejisi  4.167410e+06  4.199441e+06  32030.54341
Risk_Stratejisi     7.185610e+06  7.211430e+06  25820.43242


In [16]:
# Dengesizlik performans analizi
print("=== YANBOLU HES - Dengesizlik Performans Analizi ===")
print(f"Ortalama Dengesizlik Oranı: {(df['Dengesizlik'].sum() / df['KGUP'].sum() * 100):.2f}%")
print(f"Pozitif Dengesizlik Oranı: {(df['Dengesizlik'] > 0).sum() / len(df) * 100:.1f}%")
print(f"Negatif Dengesizlik Oranı: {(df['Dengesizlik'] < 0).sum() / len(df) * 100:.1f}%")
print(f"Sıfır Üretim Saatleri: {(df['URETIM'] == 0).sum()} saat ({(df['URETIM'] == 0).sum()/len(df)*100:.1f}%)")

# Dengesizlik büyüklük dağılımı
dengesizlik_abs = df['Dengesizlik'].abs()
print(f"\nDengesizlik Büyüklük İstatistikleri:")
print(f"Medyan: {dengesizlik_abs.median():.3f} MWh")
print(f"Ortalama: {dengesizlik_abs.mean():.3f} MWh") 
print(f"95. Persentil: {dengesizlik_abs.quantile(0.95):.3f} MWh")
print(f"Maksimum: {dengesizlik_abs.max():.3f} MWh")

=== YANBOLU HES - Dengesizlik Performans Analizi ===
Ortalama Dengesizlik Oranı: 4.07%
Pozitif Dengesizlik Oranı: 56.5%
Negatif Dengesizlik Oranı: 23.1%
Sıfır Üretim Saatleri: 2182 saat (24.8%)

Dengesizlik Büyüklük İstatistikleri:
Medyan: 0.120 MWh
Ortalama: 0.575 MWh
95. Persentil: 2.410 MWh
Maksimum: 7.310 MWh


In [18]:
def performance_nonzero(df, name):
    # Sadece üretim yapılan saatleri al
    active = df[df["URETIM"] > 0].copy()
    result = {"Santral": name, "Aktif_Saat": len(active)}

    # Tahmine göre oransal sapma
    valid = active[active["KGUP"] > 0]
    result["Ortalama_Dengesizlik_%"] = (valid["Dengesizlik"].abs() / valid["KGUP"]).mean() * 100
    result["Medyan_Dengesizlik_%"]   = (valid["Dengesizlik"].abs() / valid["KGUP"]).median() * 100

    # Ortalama bias (pozitif = fazla üretim)
    result["Bias_MWh"]     = active["Dengesizlik"].mean()
    result["Pozitif_%"]    = (active["Dengesizlik"] > 0).mean() * 100
    result["Negatif_%"]    = (active["Dengesizlik"] < 0).mean() * 100

    # Dengesizlik büyüklüğü istatistikleri
    result["Medyan_MWh"]   = active["Dengesizlik"].median()
    result["Ortalama_MWh"] = active["Dengesizlik"].mean()
    result["P95_MWh"]      = active["Dengesizlik"].quantile(0.95)
    result["Max_MWh"]      = active["Dengesizlik"].max()

    return result

yanbolu_perf = performance_nonzero(df, "YANBOLU_HES")

pd.DataFrame([yanbolu_perf]).round(3)


,Santral,Aktif_Saat,Ortalama_Dengesizlik_%,Medyan_Dengesizlik_%,Bias_MWh,Pozitif_%,Negatif_%,Medyan_MWh,Ortalama_MWh,P95_MWh,Max_MWh
0,YANBOLU_HES,6602,27.023,9.0,0.219,75.204,24.296,0.06,0.219,2.169,7.31


In [21]:
# Finansal etki analizi
print("=== YANBOLU HES - Finansal Etki Analizi ===")
total_net_revenue = df['Net_Gelir'].sum()
total_dm_cost = df['Dengesizlik_Maliyeti'].sum()
total_production = df['URETIM'].sum()

print(f"Toplam Net Gelir: {total_net_revenue:,.0f} TL")
print(f"Toplam Dengesizlik Maliyeti: {total_dm_cost:,.0f} TL")
print(f"DM'nin Net Gelir'deki Payı: {total_dm_cost/total_net_revenue*100:.2f}%")
print(f"Yıllık Birim Dengesizlik Maliyeti: {total_dm_cost/total_production:.2f} TL/MWh")

# Aylık finansal performans
monthly_financial = df_monthly[['Net_Gelir', 'Dengesizlik_Maliyeti', 'DM_Gelir_Orani']].copy()
print(f"\nEn yüksek DM/Gelir oranına sahip aylar:")
print(monthly_financial.nlargest(3, 'DM_Gelir_Orani'))

# PTF-SMF spread analizi
df['PTF_SMF_Spread'] = abs(df['PTF'] - df['SMF'])
spread_analysis = df.groupby(pd.cut(df['PTF_SMF_Spread'], bins=5)).agg({
    'Dengesizlik_Maliyeti': 'mean',
    'Birim_DM': 'mean'
}).round(2)
print(f"\nPTF-SMF Spread'e göre DM analizi:")
spread_analysis

=== YANBOLU HES - Finansal Etki Analizi ===
Toplam Net Gelir: 43,946,598 TL
Toplam Dengesizlik Maliyeti: 1,462,907 TL
DM'nin Net Gelir'deki Payı: 3.33%
Yıllık Birim Dengesizlik Maliyeti: 68.44 TL/MWh

En yüksek DM/Gelir oranına sahip aylar:
     Net_Gelir  Dengesizlik_Maliyeti  DM_Gelir_Orani
Ay                                                  
1   1616387.37             100554.50          6.2209
9   1265967.73              63497.25          5.0157
2   2225886.34             109639.00          4.9256

PTF-SMF Spread'e göre DM analizi:


/var/folders/1t/15hmc_h12xnbygjnmf7kj43h0000gn/T/ipykernel_7866/3650584721.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  spread_analysis = df.groupby(pd.cut(df['PTF_SMF_Spread'], bins=5)).agg({


,Dengesizlik_Maliyeti,Birim_DM
PTF_SMF_Spread,,
"(-2.695, 539.0]",87.60,40.26
"(539.0, 1078.0]",306.95,103.69
"(1078.0, 1617.0]",414.12,144.46
"(1617.0, 2156.0]",542.55,186.02
"(2156.0, 2695.0]",2107.60,1056.13


In [16]:
def melkom_dengesizlik_yonu_analizi(df):
    """
    YANBOLU HES için pozitif vs negatif dengesizlik analizi
    Hangi yönde hata yapmak daha pahalı?
    """
    
    print("=" * 70)
    print("YANBOLU HES - POZİTİF vs NEGATİF DENGESİZLİK ANALİZİ")
    print("=" * 70)
    
    # Dengesizlik yönlerini kategorize et
    df_analysis = df.copy()
    df_analysis['dengesizlik_yonu'] = 'Nötr'
    df_analysis.loc[df_analysis['Dengesizlik'] > 0, 'dengesizlik_yonu'] = 'Pozitif_Dengesizlik'  # Fazla ürettik
    df_analysis.loc[df_analysis['Dengesizlik'] < 0, 'dengesizlik_yonu'] = 'Negatif_Dengesizlik'  # Az ürettik
    
    # Her yön için detaylı analiz
    yön_analizi = df_analysis.groupby('dengesizlik_yonu').agg({
        'Dengesizlik': ['count', 'mean', 'std', 'min', 'max'],
        'Dengesizlik_Maliyeti': ['sum', 'mean', 'max'],
        'Birim_DM': ['mean', 'max'],
        'PTF': 'mean',
        'SMF': 'mean',
        'Poz_DF': 'mean',
        'Neg_DF': 'mean'
    }).round(2)
    
    print("\nYÖN BAZLI İSTATİSTİKLER:")
    print("-" * 50)
    
    for yön in ['Pozitif_Dengesizlik', 'Negatif_Dengesizlik', 'Nötr']:
        if yön in yön_analizi.index:
            data = yön_analizi.loc[yön]
            saat_sayisi = data[('Dengesizlik', 'count')]
            toplam_maliyet = data[('Dengesizlik_Maliyeti', 'sum')]
            ort_maliyet = data[('Dengesizlik_Maliyeti', 'mean')]
            
            if yön == 'Pozitif_Dengesizlik':
                print(f"FAZLA ÜRETİM (Tahmin > Gerçek):")
            elif yön == 'Negatif_Dengesizlik':
                print(f"EKSIK ÜRETİM (Tahmin < Gerçek):")
            else:
                print(f"MÜKEMMEL TAHMİN:")
            
            print(f"   Olay Sayısı: {saat_sayisi:.0f} saat")
            print(f"   Toplam Maliyet: {toplam_maliyet:,.0f} TL")
            print(f"   Ortalama Maliyet: {ort_maliyet:.0f} TL/saat")
            print(f"   Ortalama PTF: {data[('PTF', 'mean')]:.0f} TL/MWh")
            print(f"   Ortalama SMF: {data[('SMF', 'mean')]:.0f} TL/MWh")
            print()
    
    # Hangi yön daha maliyetli?
    pos_toplam = df_analysis[df_analysis['dengesizlik_yonu']=='Pozitif_Dengesizlik']['Dengesizlik_Maliyeti'].sum()
    neg_toplam = df_analysis[df_analysis['dengesizlik_yonu']=='Negatif_Dengesizlik']['Dengesizlik_Maliyeti'].sum()
    
    print("KRITIK BULGULAR:")
    print("-" * 30)
    print(f"Pozitif Dengesizlik Maliyeti: {pos_toplam:,.0f} TL")
    print(f"Negatif Dengesizlik Maliyeti: {neg_toplam:,.0f} TL")
    
    if neg_toplam > pos_toplam:
        risk_orani = neg_toplam / pos_toplam
        print(f"EKSIK ÜRETİM {risk_orani:.1f}x DAHA RİSKLİ!")
        print("Çünkü: SMF genelde PTF'den yüksek + %103 ceza çarpanı")
    else:
        risk_orani = pos_toplam / neg_toplam
        print(f"FAZLA ÜRETİM {risk_orani:.1f}x DAHA RİSKLİ!")
        print("Çünkü: PTF düşük dönemlerde fazla üretim zarar")
    
    # Fiyat spread analizi
    print(f"\nFİYAT SPREAD ANALİZİ:")
    print("-" * 30)
    ptf_ort = df['PTF'].mean()
    smf_ort = df['SMF'].mean()
    spread = smf_ort - ptf_ort
    
    print(f"Ortalama PTF: {ptf_ort:.0f} TL/MWh")
    print(f"Ortalama SMF: {smf_ort:.0f} TL/MWh")
    
    if spread > 0:
        print("SMF > PTF olduğu için NEGATİF dengesizlik daha riskli")
        print("   (Eksik üretimde yüksek SMF ile ceza yiyoruz)")
    else:
        print("PTF > SMF olduğu için POZİTİF dengesizlik daha riskli")
    
    # Saatlik risk analizi
    pozitif_saatlik = df_analysis[df_analysis['dengesizlik_yonu']=='Pozitif_Dengesizlik'].groupby(
        df_analysis['Saat'].str.replace(':00', '').astype(int)
    )['Dengesizlik_Maliyeti'].sum().sort_values(ascending=False)
    
    negatif_saatlik = df_analysis[df_analysis['dengesizlik_yonu']=='Negatif_Dengesizlik'].groupby(
        df_analysis['Saat'].str.replace(':00', '').astype(int)
    )['Dengesizlik_Maliyeti'].sum().sort_values(ascending=False)
    
    
    return {
        'pozitif_maliyet': pos_toplam,
        'negatif_maliyet': neg_toplam,
        'risk_yonu': 'negatif' if neg_toplam > pos_toplam else 'pozitif',
        'risk_orani': max(neg_toplam, pos_toplam) / min(neg_toplam, pos_toplam)
    }

# Analizi çalıştır
dengesizlik_analizi = melkom_dengesizlik_yonu_analizi(df)

YANBOLU HES - POZİTİF vs NEGATİF DENGESİZLİK ANALİZİ

YÖN BAZLI İSTATİSTİKLER:
--------------------------------------------------
FAZLA ÜRETİM (Tahmin > Gerçek):
   Olay Sayısı: 4965 saat
   Toplam Maliyet: 887,452 TL
   Ortalama Maliyet: 179 TL/saat
   Ortalama PTF: 2210 TL/MWh
   Ortalama SMF: 2151 TL/MWh

EKSIK ÜRETİM (Tahmin < Gerçek):
   Olay Sayısı: 2029 saat
   Toplam Maliyet: 575,455 TL
   Ortalama Maliyet: 284 TL/saat
   Ortalama PTF: 2184 TL/MWh
   Ortalama SMF: 2142 TL/MWh

MÜKEMMEL TAHMİN:
   Olay Sayısı: 1790 saat
   Toplam Maliyet: 0 TL
   Ortalama Maliyet: 0 TL/saat
   Ortalama PTF: 2366 TL/MWh
   Ortalama SMF: 2297 TL/MWh

KRITIK BULGULAR:
------------------------------
Pozitif Dengesizlik Maliyeti: 887,452 TL
Negatif Dengesizlik Maliyeti: 575,455 TL
FAZLA ÜRETİM 1.5x DAHA RİSKLİ!
Çünkü: PTF düşük dönemlerde fazla üretim zarar

FİYAT SPREAD ANALİZİ:
------------------------------
Ortalama PTF: 2236 TL/MWh
Ortalama SMF: 2179 TL/MWh
PTF > SMF olduğu için POZİTİF dengesizl

In [30]:
# Pazar dinamikleri analizi
print("=== YANBOLU HES - Pazar Dinamikleri Analizi ===")

# Yüksek/düşük fiyat dönemleri
high_price_threshold = df['PTF'].quantile(0.8)
low_price_threshold = df['PTF'].quantile(0.2)

high_price_perf = df[df['PTF'] >= high_price_threshold].agg({
    'URETIM': 'mean',
    'Dengesizlik': 'mean',
    'Dengesizlik_Maliyeti': 'mean',
    'Birim_DM': 'mean'
}).round(3)

low_price_perf = df[df['PTF'] <= low_price_threshold].agg({
    'URETIM': 'mean', 
    'Dengesizlik': 'mean',
    'Dengesizlik_Maliyeti': 'mean',
    'Birim_DM': 'mean'
}).round(3)

print(f"Yüksek Fiyat Dönemlerinde (PTF > {high_price_threshold:.0f} TL/MWh):")
print(f"Ortalama Üretim: {high_price_perf['URETIM']:.2f} MWh")
print(f"Ortalama Dengesizlik: {high_price_perf['Dengesizlik']:.3f} MWh")
print(f"Ortalama Birim DM: {high_price_perf['Birim_DM']:.2f} TL/MWh")

print(f"\nDüşük Fiyat Dönemlerinde (PTF < {low_price_threshold:.0f} TL/MWh):")
print(f"Ortalama Üretim: {low_price_perf['URETIM']:.2f} MWh")
print(f"Ortalama Dengesizlik: {low_price_perf['Dengesizlik']:.3f} MWh")
print(f"Ortalama Birim DM: {low_price_perf['Birim_DM']:.2f} TL/MWh")

=== YANBOLU HES - Pazar Dinamikleri Analizi ===
Yüksek Fiyat Dönemlerinde (PTF > 2742 TL/MWh):
Ortalama Üretim: 1.39 MWh
Ortalama Dengesizlik: 0.003 MWh
Ortalama Birim DM: 31.34 TL/MWh

Düşük Fiyat Dönemlerinde (PTF < 1800 TL/MWh):
Ortalama Üretim: 3.04 MWh
Ortalama Dengesizlik: 0.173 MWh
Ortalama Birim DM: 79.33 TL/MWh


In [53]:
# Kayıp oranı hesabı (mükemmel senaryoya göre)
print("=== YANBOLU HES - Kayıp Oranı Hesabı (Mükemmel Senaryo) ===")

# Mükemmel senaryo: Hiç dengesizlik olmasaydı kazanılacak gelir
perfect_revenue = df['URETIM'] * df['PTF']
actual_revenue = df['Net_Gelir']
revenue_loss = perfect_revenue - actual_revenue

# Kayıp oranları
total_perfect_revenue = perfect_revenue.sum()
total_actual_revenue = actual_revenue.sum()
total_revenue_loss = revenue_loss.sum()

print(f"Mükemmel Senaryo Geliri: {total_perfect_revenue:,.0f} TL")
print(f"Gerçek Net Gelir: {total_actual_revenue:,.0f} TL")
print(f"Toplam Gelir Kaybı: {total_revenue_loss:,.0f} TL")
print(f"Kayıp Oranı: {(total_revenue_loss/total_perfect_revenue)*100:.2f}%")

# Aylık kayıp oranları
monthly_loss = df.groupby('Ay').agg({
    'URETIM': 'sum',
    'PTF': lambda x: (df.loc[x.index, 'URETIM'] * df.loc[x.index, 'PTF']).sum(),
    'Net_Gelir': 'sum'
}).round(2)

monthly_loss['Perfect_Revenue'] = df.groupby('Ay').apply(lambda x: (x['URETIM'] * x['PTF']).sum())
monthly_loss['Revenue_Loss'] = monthly_loss['Perfect_Revenue'] - monthly_loss['Net_Gelir']
monthly_loss['Loss_Ratio'] = (monthly_loss['Revenue_Loss'] / monthly_loss['Perfect_Revenue'] * 100).round(2)

print(f"\nAylık Kayıp Oranları:")
print(monthly_loss[['Perfect_Revenue', 'Net_Gelir', 'Revenue_Loss', 'Loss_Ratio']])

=== YANBOLU HES - Kayıp Oranı Hesabı (Mükemmel Senaryo) ===
Mükemmel Senaryo Geliri: 45,409,505 TL
Gerçek Net Gelir: 43,946,598 TL
Toplam Gelir Kaybı: 1,462,907 TL
Kayıp Oranı: 3.22%

Aylık Kayıp Oranları:
    Perfect_Revenue   Net_Gelir  Revenue_Loss  Loss_Ratio
Ay                                                       
1      1.716942e+06  1616387.37   100554.4999        5.86
2      2.335525e+06  2225886.34   109638.9977        4.69
3      4.663634e+06  4499029.59   164604.3152        3.53
4      8.841473e+06  8573015.30   268457.9812        3.04
5      1.010250e+07  9834023.82   268476.6627        2.66
6      2.569156e+06  2507527.94    61627.7152        2.40
7      3.721648e+06  3629409.38    92238.5431        2.48
8      6.908694e+05   665137.41    25732.0026        3.72
9      1.329465e+06  1265967.73    63497.2519        4.78
10     3.468254e+06  3326218.78   142035.1358        4.10
11     2.254342e+06  2177750.43    76591.7233        3.40
12     3.715696e+06  3626244.31    89451

/var/folders/1t/15hmc_h12xnbygjnmf7kj43h0000gn/T/ipykernel_7866/3045829468.py:26: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  monthly_loss['Perfect_Revenue'] = df.groupby('Ay').apply(lambda x: (x['URETIM'] * x['PTF']).sum())


In [54]:
# Over/Under asimetrisi detaylı analizi
print("=== YANBOLU HES - Over/Under Asimetrisi Detaylı Analizi ===")

# Pozitif (Over) ve Negatif (Under) dengesizlikleri ayır
over_production = df[df['Dengesizlik'] > 0].copy()
under_production = df[df['Dengesizlik'] < 0].copy()

print(f"Over Production Saatleri: {len(over_production)} ({len(over_production)/len(df)*100:.1f}%)")
print(f"Under Production Saatleri: {len(under_production)} ({len(under_production)/len(df)*100:.1f}%)")

# Over/Under büyüklük analizi
over_stats = {
    'Count': len(over_production),
    'Avg_Size': over_production['Dengesizlik'].mean(),
    'Median_Size': over_production['Dengesizlik'].median(),
    'Max_Size': over_production['Dengesizlik'].max(),
    'Total_Volume': over_production['Dengesizlik'].sum(),
    'Avg_Cost': over_production['Dengesizlik_Maliyeti'].mean(),
    'Total_Cost': over_production['Dengesizlik_Maliyeti'].sum()
}

under_stats = {
    'Count': len(under_production),
    'Avg_Size': under_production['Dengesizlik'].mean(),
    'Median_Size': under_production['Dengesizlik'].median(),
    'Min_Size': under_production['Dengesizlik'].min(),
    'Total_Volume': under_production['Dengesizlik'].sum(),
    'Avg_Cost': under_production['Dengesizlik_Maliyeti'].mean(),
    'Total_Cost': under_production['Dengesizlik_Maliyeti'].sum()
}

asymmetry_df = pd.DataFrame({'Over': over_stats, 'Under': under_stats}).round(3)
print(f"\nOver vs Under Asimetrisi:")
print(asymmetry_df)

# Asimetri oranları
print(f"\nAsimetri Oranları:")
print(f"Hacim Asimetrisi: {abs(over_stats['Total_Volume']) / abs(under_stats['Total_Volume']):.2f}")
print(f"Maliyet Asimetrisi: {over_stats['Total_Cost'] / abs(under_stats['Total_Cost']):.2f}")
print(f"Frekans Asimetrisi: {over_stats['Count'] / under_stats['Count']:.2f}")

=== YANBOLU HES - Over/Under Asimetrisi Detaylı Analizi ===
Over Production Saatleri: 4965 (56.5%)
Under Production Saatleri: 2029 (23.1%)

Over vs Under Asimetrisi:
                    Over       Under
Count           4965.000    2029.000
Avg_Size           0.593      -1.039
Median_Size        0.110      -0.960
Max_Size           7.310         NaN
Total_Volume    2943.930   -2107.120
Avg_Cost         178.742     283.615
Total_Cost    887452.023  575454.745
Min_Size             NaN      -6.500

Asimetri Oranları:
Hacim Asimetrisi: 1.40
Maliyet Asimetrisi: 1.54
Frekans Asimetrisi: 2.45


In [57]:
# Bias & MAPE gibi tahmin doğruluğu metrikleri
print("=== YANBOLU HES - Bias & MAPE Tahmin Doğruluğu Metrikleri ===")

# Sadece üretim yapılan saatleri al
active_hours = df[df['KGUP'] > 0].copy()

# BIAS (Ortalama Hata)
bias = active_hours['Dengesizlik'].mean()
bias_percentage = (bias / active_hours['KGUP'].mean()) * 100

# MAPE (Mean Absolute Percentage Error)
mape = (active_hours['Dengesizlik'].abs() / active_hours['KGUP']).mean() * 100

# MAE (Mean Absolute Error)
mae = active_hours['Dengesizlik'].abs().mean()

# RMSE (Root Mean Square Error)
rmse = (active_hours['Dengesizlik'] ** 2).mean() ** 0.5

# Tahmin doğruluğu (accuracy)
accuracy = 1 - (active_hours['Dengesizlik'].abs().sum() / active_hours['KGUP'].sum())

print(f"BIAS (Ortalama Hata): {bias:.3f} MWh ({bias_percentage:.2f}%)")
print(f"MAPE (Ortalama Mutlak Yüzde Hata): {mape:.2f}%")
print(f"MAE (Ortalama Mutlak Hata): {mae:.3f} MWh")
print(f"RMSE (Kök Ortalama Kare Hata): {rmse:.3f} MWh")
print(f"Tahmin Doğruluğu: {accuracy*100:.1f}%")

# Aylık tahmin performansı
monthly_forecast = active_hours.groupby('Ay').agg({
    'Dengesizlik': ['mean', lambda x: x.abs().mean(), lambda x: (x.abs() / active_hours.loc[x.index, 'KGUP']).mean() * 100],
    'KGUP': 'mean'
}).round(3)

monthly_forecast.columns = ['BIAS', 'MAE', 'MAPE', 'KGUP_Mean']
print(f"\nAylık Tahmin Performansı:")
print(monthly_forecast)

=== YANBOLU HES - Bias & MAPE Tahmin Doğruluğu Metrikleri ===
BIAS (Ortalama Hata): 0.091 MWh (3.05%)
MAPE (Ortalama Mutlak Yüzde Hata): 31.54%
MAE (Ortalama Mutlak Hata): 0.705 MWh
RMSE (Kök Ortalama Kare Hata): 1.157 MWh
Tahmin Doğruluğu: 76.4%

Aylık Tahmin Performansı:
     BIAS    MAE    MAPE  KGUP_Mean
Ay                                 
1  -0.067  0.443  32.965      1.410
2   0.090  0.362  22.467      1.769
3   0.339  0.704  25.714      2.814
4   0.287  1.022  17.489      6.594
5   0.378  1.056  22.623      6.185
6  -0.141  0.570  32.178      2.291
7   0.217  0.873  49.012      2.040
8  -0.178  0.380  31.521      1.208
9  -0.073  0.684  45.163      1.523
10  0.108  0.974  59.086      2.157
11 -0.219  0.451  21.988      2.097
12 -0.102  0.526  25.004      2.668


In [70]:
# Ceza asimetrisi stratejisi (overweight optimizasyonu)
print("=== YANBOLU HES - Ceza Asimetrisi Stratejisi (Overweight Optimizasyonu) ===")

# Pozitif ve negatif dengesizliklerin ceza oranlarını analiz et
positive_imb = df[df['Dengesizlik'] > 0].copy()
negative_imb = df[df['Dengesizlik'] < 0].copy()

if len(positive_imb) > 0 and len(negative_imb) > 0:
    # Ceza oranları (Dengesizlik Tutarı / (Dengesizlik * PTF))
    positive_imb['Penalty_Rate'] = positive_imb['Dengesizlik_Tutarı'] / (positive_imb['Dengesizlik'] * positive_imb['PTF'])
    negative_imb['Penalty_Rate'] = abs(negative_imb['Dengesizlik_Tutarı']) / (abs(negative_imb['Dengesizlik']) * negative_imb['PTF'])
    
    avg_pos_penalty = positive_imb['Penalty_Rate'].mean()
    avg_neg_penalty = negative_imb['Penalty_Rate'].mean()
    
    print(f"Ortalama Pozitif Dengesizlik Ceza Oranı: {avg_pos_penalty:.3f}")
    print(f"Ortalama Negatif Dengesizlik Ceza Oranı: {avg_neg_penalty:.3f}")
    print(f"Ceza Asimetrisi (Neg/Pos): {avg_neg_penalty/avg_pos_penalty:.2f}")
    
    # Overweight stratejisi simülasyonu
    overweight_scenarios = [0.05, 0.10, 0.15, 0.20]  # %5, %10, %15, %20 fazla planlama
    
    print(f"\nOverweight Stratejisi Simülasyonu:")
    for ow in overweight_scenarios:
        sim_kgup = df['KGUP'] * (1 + ow)
        sim_dengesizlik = df['URETIM'] - sim_kgup
        
        # Yeni dengesizlik tutarları (vectorized çözüm)
        sim_dt = np.where(
            sim_dengesizlik >= 0,
            sim_dengesizlik * df['Poz_DF'],
            sim_dengesizlik * df['Neg_DF']
        )
        
        # Dengesizlik maliyeti değişimi
        original_dm = df['Dengesizlik_Maliyeti'].sum()
        sim_dm = ((df['URETIM'] * df['PTF']) - (sim_kgup * df['PTF'] + sim_dt)).clip(lower=0).sum()
        
        dm_change = sim_dm - original_dm
        dm_change_pct = (dm_change / original_dm) * 100
        
        print(f"  {ow*100:2.0f}% Overweight: DM Değişimi = {dm_change:,.0f} TL ({dm_change_pct:+.1f}%)")

# Optimal overweight oranı önerisi
print(f"\nÖneri: Negatif ceza oranı pozitiften {avg_neg_penalty/avg_pos_penalty:.1f}x yüksek")
print(f"       {5+(avg_neg_penalty/avg_pos_penalty-1)*10:.0f}% overweight stratejisi test edilebilir")

=== YANBOLU HES - Ceza Asimetrisi Stratejisi (Overweight Optimizasyonu) ===
Ortalama Pozitif Dengesizlik Ceza Oranı: 0.841
Ortalama Negatif Dengesizlik Ceza Oranı: 1.277
Ceza Asimetrisi (Neg/Pos): 1.52

Overweight Stratejisi Simülasyonu:
   5% Overweight: DM Değişimi = 2,376 TL (+0.2%)
  10% Overweight: DM Değişimi = 98,525 TL (+6.7%)
  15% Overweight: DM Değişimi = 226,312 TL (+15.5%)
  20% Overweight: DM Değişimi = 385,287 TL (+26.3%)

Öneri: Negatif ceza oranı pozitiften 1.5x yüksek
       10% overweight stratejisi test edilebilir


In [75]:
# Hidro özgü Fiyat Kullanım Endeksi
print("=== YANBOLU HES - Hidro Özgü Fiyat Kullanım Endeksi ===")

# Fiyat seviyelerine göre üretim verimliliği (geçici değişkenler kullan)
price_quintiles = df['PTF'].quantile([0.2, 0.4, 0.6, 0.8])
price_levels = pd.cut(df['PTF'], 
                     bins=[0, price_quintiles[0.2], price_quintiles[0.4], price_quintiles[0.6], price_quintiles[0.8], float('inf')],
                     labels=['Çok_Düşük', 'Düşük', 'Orta', 'Yüksek', 'Çok_Yüksek'])

# Her fiyat seviyesinde üretim davranışı
price_efficiency = df.groupby(price_levels).agg({
    'PTF': 'mean',
    'URETIM': ['mean', 'sum', 'count'],
    'KGUP': ['mean', 'sum'],
    'Dengesizlik_Maliyeti': ['mean', 'sum']
}).round(2)

price_efficiency.columns = ['Avg_PTF', 'Avg_Production', 'Total_Production', 'Hour_Count', 'Avg_KGUP', 'Total_KGUP', 'Avg_DM', 'Total_DM']

# Fiyat Kullanım Endeksi hesapla
total_production = df['URETIM'].sum()
weighted_price = (price_efficiency['Total_Production'] * price_efficiency['Avg_PTF']).sum() / total_production
average_price = df['PTF'].mean()
price_utilization_index = weighted_price / average_price

print(f"Fiyat Seviyelerine Göre Üretim Dağılımı:")
print(price_efficiency[['Avg_PTF', 'Total_Production', 'Hour_Count', 'Avg_DM']])

print(f"\nFiyat Kullanım Endeksi: {price_utilization_index:.3f}")
print(f"(1.0'dan yüksek = yüksek fiyatlarda daha çok üretim)")

# Su rejimi stratejisi analizi
daily_production = df.groupby('Tarih').agg({
    'URETIM': 'sum',
    'PTF': 'mean',
    'Dengesizlik_Maliyeti': 'sum'
}).reset_index()

# Günlük üretim vs fiyat korelasyonu
daily_corr = daily_production['URETIM'].corr(daily_production['PTF'])
print(f"\nGünlük Üretim-Fiyat Korelasyonu: {daily_corr:.3f}")

# En değerli günlerde üretim performansı
top_price_days = daily_production.nlargest(30, 'PTF')
avg_production_high_price = top_price_days['URETIM'].mean()
avg_production_overall = daily_production['URETIM'].mean()
high_price_utilization = avg_production_high_price / avg_production_overall

print(f"En Yüksek Fiyatlı 30 Günde Ort. Üretim: {avg_production_high_price:.1f} MWh")
print(f"Genel Ortalama Günlük Üretim: {avg_production_overall:.1f} MWh")
print(f"Yüksek Fiyat Kullanım Oranı: {high_price_utilization:.2f}")

# Hidro strateji skoru
hydro_strategy_score = (price_utilization_index + daily_corr + high_price_utilization) / 3 * 100
print(f"\nHidro Strateji Skoru: {hydro_strategy_score:.1f}/100")
print(f"(Fiyat takibi, günlük korelasyon ve yüksek fiyat kullanımının ortalaması)")

=== YANBOLU HES - Hidro Özgü Fiyat Kullanım Endeksi ===
Fiyat Seviyelerine Göre Üretim Dağılımı:
            Avg_PTF  Total_Production  Hour_Count  Avg_DM
PTF                                                      
Çok_Düşük   1161.62           5306.17        1754  251.93
Düşük       2076.86           3972.69        1756  189.26
Orta        2374.00           4893.02        1758  174.97
Yüksek      2629.05           4726.61        1755  147.39
Çok_Yüksek  2939.62           2447.19        1757   69.67

Fiyat Kullanım Endeksi: 0.955
(1.0'dan yüksek = yüksek fiyatlarda daha çok üretim)

Günlük Üretim-Fiyat Korelasyonu: -0.331
En Yüksek Fiyatlı 30 Günde Ort. Üretim: 35.0 MWh
Genel Ortalama Günlük Üretim: 58.4 MWh
Yüksek Fiyat Kullanım Oranı: 0.60

Hidro Strateji Skoru: 40.8/100
(Fiyat takibi, günlük korelasyon ve yüksek fiyat kullanımının ortalaması)


/var/folders/1t/15hmc_h12xnbygjnmf7kj43h0000gn/T/ipykernel_7866/727018244.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  price_efficiency = df.groupby(price_levels).agg({
